# https://www.kaggle.com/c/digit-recognizer/
# with keras + CNN, scored 0.9785714285714285

In [60]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

In [61]:
# load csv files to numpy arrays

def load_data():
    train = pd.read_csv("train.csv")
    X_train = train.values[:,1:] 
    y_train = train.values[:,0]
    
    Pred_test = pd.read_csv("test.csv").values
    return X_train, y_train, Pred_test

def flush_predictons(predictions, output_name='out.csv'):
    '''
    predictions is a np.array of int 
    '''
    imgs = pd.Series(range(1,pred_test.shape[0]+1))
    imgd = pd.DataFrame(imgs)
    
    labeld = pd.DataFrame(predictions)
    out = pd.concat([imgd, labeld],axis=1)
    out.columns = ["ImageId", "Label"]
    out.to_csv(output_name, index = False)
    
x_train, y_train, pred_test = load_data()

In [62]:
print(x_train.shape, y_train.shape, pred_test.shape)

(42000, 784) (42000,) (28000, 784)


In [63]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, train_size=0.8, random_state=0)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(33600, 784) (33600,) (8400, 784) (8400,)


In [64]:
plt.imshow(x_train[4].reshape(28, 28))

In [65]:
from keras import models, layers
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten, ZeroPadding2D, BatchNormalization
from keras.optimizers import RMSprop

In [66]:
# data preprocessing for keras CNN
x_train = x_train.reshape(x_train.shape[0], 28,28,1).astype('float32')/255
x_test  = x_test.reshape(x_test.shape[0], 28,28,1).astype('float32')/255

In [67]:
batch_size = 128
num_classes = 10
epochs = 20
pool_size=(2,2)

In [68]:
# one-hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [69]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(33600, 28, 28, 1) (33600, 10) (8400, 28, 28, 1) (8400, 10)


In [71]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1,)))
model.add(Activation('relu'))
model.add(BatchNormalization(epsilon=1e-6))
model.add(MaxPool2D(pool_size=pool_size))

model.add(ZeroPadding2D(padding=(1,1)))
model.add(Conv2D(48, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization(epsilon=1e-6))
model.add(MaxPool2D(pool_size=pool_size))

model.add(ZeroPadding2D(padding=(1,1)))
model.add(Conv2D(64, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization(epsilon=1e-6))
model.add(MaxPool2D(pool_size=pool_size))

model.add(ZeroPadding2D(padding=(1,1)))
model.add(Conv2D(80, kernel_size=(2,2)))
model.add(Activation('relu'))
model.add(BatchNormalization(epsilon=1e-6))
model.add(MaxPool2D(pool_size=pool_size))


model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(2560))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))
 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
activation_21 (Activation)   (None, 26, 26, 32)        0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
zero_padding2d_9 (ZeroPaddin (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 13, 13, 48)        13872     
_________________________________________________________________
activation_22 (Activation)   (None, 13, 13, 48)        0         
__________

In [72]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=80,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 33600 samples, validate on 8400 samples
Epoch 1/80
33600/33600 [==============================] - 53s 2ms/step - loss: 0.5557 - acc: 0.9129 - val_loss: 0.1081 - val_acc: 0.9679
Epoch 2/80
33600/33600 [==============================] - 51s 2ms/step - loss: 0.0754 - acc: 0.9782 - val_loss: 0.1348 - val_acc: 0.9663
Epoch 3/80
33600/33600 [==============================] - 51s 2ms/step - loss: 0.0534 - acc: 0.9849 - val_loss: 0.0995 - val_acc: 0.9773
Epoch 4/80
33600/33600 [==============================] - 51s 2ms/step - loss: 0.0481 - acc: 0.9877 - val_loss: 0.0671 - val_acc: 0.9831
Epoch 5/80
33600/33600 [==============================] - 51s 2ms/step - loss: 0.0386 - acc: 0.9896 - val_loss: 0.0736 - val_acc: 0.9839
Epoch 6/80
33600/33600 [==============================] - 52s 2ms/step - loss: 0.0332 - acc: 0.9914 - val_loss: 0.0632 - val_acc: 0.9876
Epoch 7/80
33600/33600 [==============================] - 51s 2ms/step - loss: 0.0306 - acc: 0.9924 - val_loss: 0.0845 - val_acc:

33600/33600 [==============================] - 3s 88us/step - loss: 0.0036 - acc: 0.9992 - val_loss: 0.0934 - val_acc: 0.9906
Epoch 61/80
33600/33600 [==============================] - 3s 88us/step - loss: 0.0053 - acc: 0.9990 - val_loss: 0.0775 - val_acc: 0.9920
Epoch 62/80
33600/33600 [==============================] - 3s 87us/step - loss: 0.0047 - acc: 0.9992 - val_loss: 0.0964 - val_acc: 0.9915
Epoch 63/80
33600/33600 [==============================] - 3s 88us/step - loss: 0.0045 - acc: 0.9992 - val_loss: 0.0878 - val_acc: 0.9924
Epoch 64/80
33600/33600 [==============================] - 3s 88us/step - loss: 0.0067 - acc: 0.9990 - val_loss: 0.0923 - val_acc: 0.9908
Epoch 65/80
33600/33600 [==============================] - 3s 89us/step - loss: 0.0069 - acc: 0.9990 - val_loss: 0.0931 - val_acc: 0.9908
Epoch 66/80
33600/33600 [==============================] - 3s 88us/step - loss: 0.0103 - acc: 0.9987 - val_loss: 0.0802 - val_acc: 0.9923
Epoch 67/80
33600/33600 [=====================

In [73]:
pred_test = pred_test.reshape(pred_test.shape[0], 28,28,1).astype('float32')/255
pred = model.predict_classes(pred_test)

In [74]:
flush_predictons(pred, output_name='keras_cnn2.csv')

In [75]:
model.save("keras_cnn2.h5")

In [ ]:
#from keras.models import load_model

#model = load_model("keras_cnn.h5")